## Test dữ liệu video

In [10]:
import cv2
from ultralytics import YOLO

# Đường dẫn tới model và video (chú ý sử dụng r"" để tránh vấn đề escape)
model_path = r"D:\USB_MSI\ClassRoom\IT_Semester2_20242025\04_AI_IOT\FinalPorject\YOLO-Behavior-Student\Server\app\model-weights\yolov12s13032025_234132_ver-dataset6\_ver2\runs\detect\train\weights\best.pt"
video_path = r"D:\USB_MSI\ClassRoom\IT_Semester2_20242025\04_AI_IOT\FinalPorject\YOLO-Behavior-Student\13.00.40-13.05.00[M][0@0][0] ver3.mp4"

# Load model YOLO
model = YOLO(model_path)

# Mở video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Không thể mở video!")
    exit()

# Lấy thông số video
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output_detected.mp4", fourcc, fps, (width, height))

# In ra kích thước frame (image size)
print(f"Frame size: {width} x {height} và FPS: {fps}")


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chạy inference trên frame (verbose=False để không in log quá nhiều)
    results = model(frame, verbose=False)
    
    # Vòng lặp qua các kết quả (một frame có thể có nhiều kết quả nếu sử dụng mosaic,...)
    for result in results:
        # result.boxes chứa các bounding box dự đoán
        boxes = result.boxes
        if boxes is not None:
            # Với mỗi box: lấy toạ độ (xyxy), confidence và class id
            for box in boxes:
                # Lấy toạ độ dưới dạng float (xyxy)
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                # Lấy tên lớp từ model (model.names)
                label = f"{model.names[cls]} {conf:.2f}"
                
                # Vẽ bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # Vẽ label ở trên box
                cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Hiển thị frame với kết quả detect
    cv2.imshow("Detection", frame)
    out.write(frame)
    
    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Frame size: 1920 x 1080 và FPS: 19.854445023373767


## Cải thiện lại khả năng đọc frame, xử lý suy luận, cải thiện FPS

In [19]:
import cv2
from ultralytics import YOLO
import torch
import threading
from queue import Queue

# ------------------- CẤU HÌNH -------------------
# Đường dẫn tới model và video (chú ý sử dụng raw string r"" để tránh lỗi escape)
model_path = r"D:\USB_MSI\ClassRoom\IT_Semester2_20242025\04_AI_IOT\FinalPorject\YOLO-Behavior-Student\Server\app\model-weights\yolov12s13032025_234132_ver-dataset6\_ver2\runs\detect\train\weights\best.pt"
video_path = r"D:\USB_MSI\ClassRoom\IT_Semester2_20242025\04_AI_IOT\FinalPorject\YOLO-Behavior-Student\13.00.40-13.05.00[M][0@0][0] ver3.mp4"

# Kích thước đầu vào cho inference sau khi resize
imgsz = (1920, 1080)
batch_size = 4
half_precision = True if torch.cuda.is_available() else False

# ------------------- KHỞI TẠO MODEL -------------------
model = YOLO(model_path)
model.fuse()  # Ghép các lớp để tăng tốc inference
if half_precision:
    model.half()  # Sử dụng half precision nếu có GPU hỗ trợ

# ------------------- HÀNG ĐỢI -------------------
input_queue = Queue(maxsize=10)
output_queue = Queue(maxsize=10)

# ------------------- WORKER CHO INFERENCE -------------------
def inference_worker():
    while True:
        frames = []
        # Lấy đủ số lượng frame theo batch_size từ hàng đợi input
        while len(frames) < batch_size and not input_queue.empty():
            frames.append(input_queue.get())
        
        if frames:
            with torch.no_grad():
                # Chạy inference với kích thước ảnh đã được resize (imgsz)
                results = model(frames, imgsz=imgsz, verbose=False, half=half_precision)
            # Đưa ảnh gốc đã được vẽ bounding box vào hàng đợi output
            for res in results:
                output_queue.put(res.orig_img)

# Khởi tạo thread worker cho inference (daemon=True để tự động kết thúc khi chương trình dừng)
threading.Thread(target=inference_worker, daemon=True).start()

# ------------------- ĐỌC VIDEO -------------------
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Không thể mở video!")
    exit()

# Lấy thông số video gốc (không cần dùng thông số này nếu chúng ta định resize tất cả frame)
# Nhưng chúng ta có thể in ra để kiểm tra
orig_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
orig_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps         = cap.get(cv2.CAP_PROP_FPS)
print(f"Video gốc: {orig_width} x {orig_height} - FPS: {fps}")

# ------------------- OUTPUT VIDEO -------------------
# Sửa kích thước output video thành imgsz (1920, 1080) để khớp với frame đã resize
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, imgsz)

# ------------------- VÒNG LẶP XỬ LÝ -------------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize frame về kích thước được định nghĩa (imgsz = (1920, 1080))
    resized_frame = cv2.resize(frame, imgsz)  # CHÚ THÍCH: Resize frame theo kích thước imgsz
    # Đưa frame đã resize vào hàng đợi input (nếu hàng đợi chưa đầy)
    if input_queue.qsize() < 10:
        input_queue.put(resized_frame)
    
    # Nếu có kết quả từ output queue, lấy và ghi vào video output
    if not output_queue.empty():
        processed_frame = output_queue.get()
        out.write(processed_frame)
        cv2.imshow("Output", processed_frame)
    
    # Nhấn 'q' để thoát
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


YOLOv8s summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs
Video gốc: 1920 x 1080 - FPS: 19.854445023373767
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to 

WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
WARNING  imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]


## Code sạch và
Quản lý tài nguyên tốt hơn:

Sử dụng khối try-except để xử lý lỗi

Đảm bảo giải phóng tài nguyên với khối finally

Thêm progress bar với tqdm để theo dõi tiến độ

## Tối ưu hiệu năng:

Tự động phát hiện và sử dụng GPU nếu có

Sử dụng model.predict() với tham số stream=True để xử lý video hiệu quả

Thêm ngưỡng confidence và IoU để lọc kết quả
## Code sạch và dễ bảo trì:

Sử dụng constant variables cho các tham số

Tách logic chính thành hàm main()

Sử dụng phương thức render tích hợp của YOLO

In [26]:
import cv2
import torch
from ultralytics import YOLO
from tqdm import tqdm

# Constants
MODEL_PATH = r"/hdd2/minhnv/CodingYOLOv12/Behavior-Detect-Student-YOLO/StaticModels/yolov12s_29032025_013131_exp1_baseline/runs/detect/train/weights/best.pt"
VIDEO_PATH = r"/hdd2/minhnv/CodingYOLOv12/TestingVideo/13.00.40-13.05.00[M][0@0][0] ver2.mp4"

OUTPUT_PATH = "output_detected_optimized.mp4"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CONF_THRESH = 0.3  # Giảm ngưỡng confidence để hiển thị nhiều đối tượng hơn
IOU_THRESH = 0.45

# Custom display settings
BOX_THICKNESS = 1      # Giảm độ dày khung
TEXT_SCALE = 0.5       # Giảm kích thước chữ
TEXT_THICKNESS = 1    # Độ dày nét vẽ chữ
COLOR = (0, 255, 0)    # Màu xanh lá

def main():
    try:
        model = YOLO(MODEL_PATH).to(DEVICE)
        print(f"Model loaded on {DEVICE.upper()}")
    except Exception as e:
        print(f"Lỗi tải model: {e}")
        return

    try:
        cap = cv2.VideoCapture(VIDEO_PATH)
        if not cap.isOpened():
            raise ValueError("Không mở được video")
    except Exception as e:
        print(f"Lỗi video: {e}")
        return

    # Lấy thông số video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))
    
    print(f"Xử lý video: {width}x{height} @ {fps:.2f} FPS")
    print(f"Tổng số frame: {total_frames:,}")

    pbar = tqdm(total=total_frames, unit='frame', desc='Processing')

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Chạy inference với threshold thấp
            results = model.predict(
                frame,
                conf=CONF_THRESH,
                iou=IOU_THRESH,
                device=DEVICE,
                verbose=False,
                stream=True
            )

            for result in results:
                # Vẽ bounding box tùy chỉnh
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    conf = box.conf[0].item()
                    cls_id = int(box.cls[0].item())
                    
                    # Vẽ bounding box
                    cv2.rectangle(frame, 
                                (x1, y1), 
                                (x2, y2), 
                                COLOR, 
                                BOX_THICKNESS)
                    
                    # Vẽ label
                    label = f"{model.names[cls_id]} {conf:.2f}"
                    (tw, th), _ = cv2.getTextSize(label,
                                                 cv2.FONT_HERSHEY_SIMPLEX,
                                                 TEXT_SCALE,
                                                 TEXT_THICKNESS)
                    
                    cv2.rectangle(frame,
                                (x1, y1 - 20),
                                (x1 + tw, y1),
                                COLOR,
                                -1)
                    
                    cv2.putText(frame,
                               label,
                               (x1, y1 - 5),
                               cv2.FONT_HERSHEY_SIMPLEX,
                               TEXT_SCALE,
                               (64, 64, 64),  # Chữ màu trắng
                               TEXT_THICKNESS)

                # Ghi frame vào output
                out.write(frame)
                
                # Hiển thị preview
                preview = cv2.resize(frame, (1280, 720))
                cv2.imshow('Detection Preview', preview)

            pbar.update(1)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"Lỗi xử lý: {e}")
    finally:
        pbar.close()
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("\nHoàn thành! Output:", OUTPUT_PATH)

if __name__ == "__main__":
    main()

Model loaded on CPU
Xử lý video: 1920x1080 @ 19.85 FPS
Tổng số frame: 824


Processing:   7%|▋         | 60/824 [00:11<02:29,  5.12frame/s]



Hoàn thành! Output: output_detected_optimized.mp4


KeyboardInterrupt: 